In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import os
import numpy as np
import pandas as pd
import time
import torch
import pickle
import shap
import tensorboard


from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from functools import partial
import shutil
from tqdm.auto import tqdm

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

from codes.models.data_form.DataForm import DataTransfo_1SNP
from codes.models.metrics import calculate_roc_auc

import featurewiz as gwiz

import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

from codes.models.Decision_tree.utils import get_indice, get_name
from pygam import GAM, s, f


In [ ]:
### framework constants:
model_type = 'decision_tree'
model_version = 'gradient_boosting'
test_name = '1_test_train_transfo_V1'
tryout = True # True if we are ding a tryout, False otherwise 
### data constants:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Paul' # Paul, Abby
ld = 'no'
rollup_depth = 4
binary_classes = True #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = False
save_data = True
remove_none = True
decorelate = False
equalize_label = False
threshold_corr = 0.9
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = False
list_env_features = ['age', 'sex']
list_pheno_ids = None #list(np.load(f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/phewas/list_associations_snps/{SNP}_paul.npy'))

### data format

batch_size = 20
data_share = 1

##### model constants


##### training constants
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=remove_none,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share,
                         list_pheno_ids=list_pheno_ids,
                         binary_classes=binary_classes, 
                         ld = ld)
#patient_list = dataT.get_patientlist()

In [ ]:
data, labels_patients, indices_env, name_envs, eids = dataT.get_tree_data(with_env=False, load_possible=True, only_relevant=False)

In [ ]:
equalized = True
interest = False
keep = False
scaled = True
remove = False

In [ ]:
if interest:
    data_use, labels_use = data[:nb_patients_interest, :-1], labels_patients[:nb_patients_interest]
else:
    data_use, labels_use = data, labels_patients
if remove:
    eids_remove = np.load('/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/UKBB/eids_remove_1.npy')
    indices_eids = (1-np.isin(eids, eids_remove)).astype(bool)
    eids_use = eids[indices_eids]
    data_use = data_use[indices_eids]
    labels_use = labels_use[indices_eids]
    
if equalized:
    pheno, labels = DataTransfo_1SNP.equalize_label(data=data_use, labels = labels_use)
else:
    pheno, labels = data_use, labels_use


In [ ]:
diseases_patients_train, diseases_patients_test, label_patients_train, label_patients_test = train_test_split(pheno_train, labels_train, test_size = 1-prop_train_test, random_state=42)

In [ ]:
class_weight = {0: np.sum(label_patients_train == 1) / np.sum(label_patients_train == 0), 1: 1.0}


In [ ]:
indices_keep = (frequencies_ini > 0) & (frequencies_ini > 100)
#indices_keep = shaps!=0

In [ ]:
diseases_patients_train_keep = diseases_patients_train[:,indices_keep]
diseases_patients_test_keep = diseases_patients_test[:, indices_keep]

In [ ]:
if keep:
    diseases_patients_train_model = diseases_patients_train_keep
    diseases_patients_test_model = diseases_patients_test_keep
else:
    diseases_patients_train_model = diseases_patients_train
    diseases_patients_test_model = diseases_patients_test



In [ ]:
    
diseases_patients_train_model_unscaled = diseases_patients_train_model
diseases_patients_test_model_unscaled = diseases_patients_test_model

if scaled:

    scaler = StandardScaler()
    diseases_patients_train_model= scaler.fit_transform(diseases_patients_train_model)
    diseases_patients_test_model = scaler.fit_transform(diseases_patients_test_model)



In [ ]:
model = HistGradientBoostingClassifier(class_weight=class_weight)


# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(diseases_patients_train_model, label_patients_train)

# Faire des prédictions sur l'ensemble de test
labels_pred_test = model.predict(diseases_patients_test_model)
labels_pred_train = model.predict(diseases_patients_train_model)
proba_test = model.predict_proba(diseases_patients_test_model)[:, 1]
proba_train = model.predict_proba(diseases_patients_train_model)[:, 1]